## First we need all necessary imports

In [1]:
import os
import numpy as np
import flopy.modflow as mf
import shutil
import geojson
import pyproj

* The modflow-files will be stored in './workspace' 
* Each calculation will delete the files

In [2]:
workspace = os.path.join('workspace')

if os.path.exists(workspace):
    shutil.rmtree(workspace)
    
if not os.path.exists(workspace):
    os.makedirs(workspace)


In [3]:
import json
f = open('configuration.json')
configuration = json.load(f)
f.close()

modflow_data = configuration['data']['mf']
modflow_package_list = modflow_data['packages']

modflow_package_list

['mf', 'bas', 'dis', 'lpf', 'pcg', 'oc', 'chd', 'lak']

In [4]:
from MfPackages import MfAdapter
mf_data_obj = modflow_data['mf']
mf_data_obj['modelname'] = 'lak_test'
mf_data_obj['exe_name'] = '../modflow/mf2005'
mf_data_obj['model_ws'] = workspace
mfAdapter = MfAdapter(mf_data_obj)
mfAdapter.merge()
model = mfAdapter.get_package()

In [5]:
from MfPackages import DisAdapter
disAdapter = DisAdapter(modflow_data['dis'])
disAdapter.merge()
dis = disAdapter.get_package(model)
dis


    MODFLOW Discretization Package Class.

    Parameters
    ----------
    model : model object
        The model object (of type :class:`flopy.modflow.Modflow`) to which
        this package will be added.
    nlay : int
        Number of model layers (the default is 1).
    nrow : int
        Number of model rows (the default is 2).
    ncol : int
        Number of model columns (the default is 2).
    nper : int
        Number of model stress periods (the default is 1).
    delr : float or array of floats (ncol), optional
        An array of spacings along a row (the default is 1.0).
    delc : float or array of floats (nrow), optional
        An array of spacings along a column (the default is 0.0).
    laycbd : int or array of ints (nlay), optional
        An array of flags indicating whether or not a layer has a Quasi-3D
        confining bed below it. 0 indicates no confining bed, and not zero
        indicates a confining bed. LAYCBD for the bottom layer must be 0. (the
    

In [6]:
from MfPackages import BasAdapter
basAdapter = BasAdapter(modflow_data['bas'])
basAdapter.merge()
bas = basAdapter.get_package(model)

In [7]:
from MfPackages import ChdAdapter
chdAdapter = ChdAdapter(modflow_data['chd'])
chdAdapter.merge()
chd = chdAdapter.get_package(model)

In [8]:
from MfPackages import LakAdapter
lakAdapter = LakAdapter(modflow_data['lak'])
lakAdapter.merge()
lak = lakAdapter.get_package(model)

In [9]:
from MfPackages import LpfAdapter
lpfAdapter = LpfAdapter(modflow_data['lpf'])
lpfAdapter.merge()
lpf = lpfAdapter.get_package(model)

In [10]:
from MfPackages import PcgAdapter
pcgAdapter = PcgAdapter(modflow_data['pcg'])
pcgAdapter.merge()
pcg = pcgAdapter.get_package(model)

In [11]:
from MfPackages import OcAdapter
ocAdapter = OcAdapter(modflow_data['oc'])
ocAdapter.merge()
oc = ocAdapter.get_package(model)

### Calculate the model

In [12]:
model.write_input()
model.run_model()

FloPy is using the following executable to run the model: ../../modflow/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: lak_test.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2023/08/18  9:29:33

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Solving:  Stress period:     2    Time step:     1    Ground-Water Flow Eqn.
 Solving:  Stress period:     3    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2023/08/18  9:29:33
 Elapsed run time:  0.002 Seconds

  Normal termination of simulation


(True, [])

### Read calculated data

In [15]:
import flopy.utils as fu

hds = fu.HeadFile(os.path.join(workspace, 'lak_test.hds'))
h = hds.get_data(kstpkper=(0,0))
h

array([[[ 100.      ,   99.11863 ,   98.20165 ,   97.21149 ,
           96.12289 ,   94.99999 ,   93.87709 ,   92.78849 ,
           91.798325,   90.88136 ,   90.      ],
        [ 100.      ,   99.201004,   98.371086,   97.43926 ,
           96.202415,   94.99999 ,   93.79757 ,   92.560715,
           91.6289  ,   90.79899 ,   90.      ],
        [ 100.      ,   99.35492 ,   98.77635 ,   98.37291 ,
         -999.99    , -999.99    , -999.99    ,   91.627075,
           91.22363 ,   90.64507 ,   90.      ],
        [ 100.      ,   99.35492 ,   98.77635 ,   98.37291 ,
         -999.99    , -999.99    , -999.99    ,   91.627075,
           91.22363 ,   90.64507 ,   90.      ],
        [ 100.      ,   99.201004,   98.371086,   97.43926 ,
           96.202415,   94.99999 ,   93.79757 ,   92.560715,
           91.6289  ,   90.79899 ,   90.      ],
        [ 100.      ,   99.11863 ,   98.20165 ,   97.21149 ,
           96.12289 ,   94.99999 ,   93.87709 ,   92.78849 ,
           91.798325,  